In [0]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics

# This is your student key that I emailed to you at the beginnning of the semester.


# You must also identify your source file.  (modify for your local setup)
# file='/resources/t81_558_deep_learning/assignment_yourname_class1.ipynb'  # IBM Data Science Workbench
# file='C:\\Users\\jeffh\\projects\\t81_558_deep_learning\\t81_558_class1_intro_python.ipynb'  # Windows

# file = "d:\\Users\\jheaton\\Dropbox\\school\\teaching\\wustl\\classes\\T81_558_deep_learning\\solutions\\assignment_solution_class9.ipynb"

# Begin assignment
#df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/datasets/reg-33-data.csv")

In [0]:
# Begin assignment
df = pd.read_csv("reg-33-data.csv")

# Encode the feature vector
ids = df['id']
df.drop('id',1,inplace=True)
df.head(5)

In [0]:
df_test = pd.read_csv('reg-33-eval.csv')
ids = df_test['id']
df_test.drop('id',1,inplace=True)
df_test.head(5)

In [0]:
numerical_cols = ['height','max','number','length','power','weight']
text_cols = ['convention','cat2','usage','region','code','item','country']

In [0]:
#fill nan with mean vlaue
for column in list(df.columns[df.isnull().sum() > 0]):
    mean_val = df[column].mean()
    df[column].fillna(mean_val, inplace=True)
    
#fill nan with mean vlaue
for column in list(df_test.columns[df_test.isnull().sum() > 0]):
    mean_val = df_test[column].mean()
    df_test[column].fillna(mean_val, inplace=True)

In [0]:
#normalize
for col in numerical_cols:
    df.loc[:,col] = zscore(df[col].values)

#normalize
for col in numerical_cols:
    df_test.loc[:,col] = zscore(df_test[col].values)

In [0]:
for col in text_cols:
    #print(col)
    df = df.join(pd.get_dummies(df[col],prefix=col))
new_df = df.drop(columns=text_cols)


for col in text_cols:
    #print(col)
    df_test = df_test.join(pd.get_dummies(df_test[col],prefix=col))
new_df_test = df_test.drop(columns=text_cols)

In [0]:
train = new_df[[x for x in list(new_df.columns) if x != 'target']].values
test = new_df['target'].values

print(train.shape, test.shape)

test_data = new_df_test[[x for x in list(new_df_test.columns) if x != 'target']].values


print(test_data.shape)

In [0]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.optimizers import SGD

def model():
    model=Sequential()
    model.add(Dense(1000, activation='relu', input_dim=256))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    adam = Adam(learning_rate=0.005, beta_1=0.9, beta_2=0.999, amsgrad=True)
    ada = Adadelta(learning_rate=0.002, rho=0.9)
    sgd = SGD(lr=0.002, decay=1e-3, momentum=0.9, nesterov=True)
    model.compile(loss='mse', optimizer=adam)
    return model

In [0]:
model1 = model()
model1.fit(train, test, batch_size=1024, epochs=1000, verbose=1)

In [0]:
predictions = model1.predict(test_data)
predictoins = predictions.reshape(-1)
sub = np.c_[np.arange(1,1001),predictions].astype(np.int32)
dataframe = pd.DataFrame(data=sub,columns=['id','target'])

In [0]:
dataframe